In [1]:
!pip install opencv-python
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
!pip install mediapipe

In [2]:
!pip install opencv-python transformers accelerate
!pip install gradio
!pip install asyncio

In [3]:
import time
t = str(int(time.time()))+"_"
t

'1728803034_'

In [4]:
from diffusers import AutoencoderKL, StableDiffusionXLControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import os

from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers import DDIMScheduler, EulerAncestralDiscreteScheduler
from controlnet_aux import MidasDetector

from PIL import Image
import torch
import random
import numpy as np
import cv2
import gradio as gr
import asyncio

2024-10-13 07:04:00.518536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 07:04:00.518617: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 07:04:00.518693: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
async def image_generator(path, amount, height, width, prompt, negative_prompt):
    #TODO: сделать количество картинoк зависимым от amount
    controlnet_conditioning_scale = 1.0

    controlnet = ControlNetModel.from_pretrained(
        "xinsir/controlnet-depth-sdxl-1.0",
        torch_dtype=torch.float16, cache_dir='./modele cache/'
    )

    # when test with other base model, you need to change the vae also.
    vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16,
                                        cache_dir='./modele cache/')

    eulera_scheduler = EulerAncestralDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
                                                                       subfolder="scheduler",
                                                                       cache_dir='./modele cache/')

    pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
        "John6666/realism-by-stable-yogi-sdxlv2-sdxl",
        controlnet=controlnet,
        vae=vae,
        torch_dtype=torch.float16,
        scheduler=eulera_scheduler,
        cache_dir='./modele cache/'
    ).to("cuda")
    for i in range(amount):
        # Получаем случайный файл маски
        mask_files = [f for f in os.listdir('./masks/') if f.endswith((".jpg", ".png"))]
        random_mask_file = random.choice(mask_files)
        controlnet_img = Image.open('./masks/' + random_mask_file).convert("L")  # Преобразуем в черно-белый формат

        # Генерация изображения с pipe
        images = pipe(
            prompt,
            negative_prompt=negative_prompt,
            image=controlnet_img,
            controlnet_conditioning_scale=controlnet_conditioning_scale,
            width=width,
            height=height,
            num_inference_steps=50,
            num_images_per_prompt=1
        )

        # Конвертируем маску в NumPy массив для применения
        controlnet_mask = np.array(controlnet_img)

        # Пройдем по всем сгенерированным изображениям
        for im in range(len(images.images)):
            # Сохраняем изображение
            # masked_pil_image.save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
            images.images[im].save(
                path+"/img/" + t + "image_" + random_mask_file[:-4] + "_" + str(im) + '_' + str(i) + ".jpg")

        # Очищаем кэш после каждой итерации
        torch.cuda.empty_cache()


async def prompt_generator(output_path, amount, height, width, race, cleanliness, jewelry, tatoo, gender, age, background):
    parameters = [race, cleanliness, jewelry, tatoo, gender, age]
    for parameter in parameters:
        if parameter is None:
            return "Error! Fullfill all fields"
    if cleanliness=="dirty":
        prompt = "A detailed, high-quality photograph, high-quality  realistically, well lit, of a dirty human palm of an "+age+" " + race+" "+gender+" "+tatoo+" "+jewelry+ " on a pure white background. "+background+" background.The palm is facing the viewer, fingers extended, with the pattern of creases and lines on the hand clearly visible. There are no other objects or elements in the image. Resolution "+str(width)+"x"+str(height)
    else:
        prompt = "A detailed, high-quality photograph, high-quality  realistically, well lit, of a human palm of an " + age + " " + race + " " + gender + " " + tatoo + " " + jewelry + " on a pure white background. White background.The palm is facing the viewer, fingers extended, with the pattern of creases and lines on the hand clearly visible. There are no other objects or elements in the image. Resolution " + str(width) + "x" + str(height)
    negative_prompt = "not a palm, drawn, cartoons, animation, drawing, Blurriness, (many folds), low resolution, other body parts, extraneous objects, background elements, text, watermarks, multiple hands, gloves, nail polish, distracting details."
    image_generator(output_path, amount, height, width, prompt, negative_prompt)
    return "Prompt: "+prompt+"\nNegative prompt: "+negative_prompt+"\nPictures are generating. Please check the output path."

demo = gr.Interface(
    prompt_generator,
    [
        gr.Textbox(value="/generated_data"),
        gr.Number(value=1000),
        gr.Number(value=512),
        gr.Number(value=512),
        gr.Radio(["european", "african", "asian"], value="european"),
        gr.Radio(["dirty", "clean"], value="clean"),
        gr.Radio(["with jewelry", "without jewelry"], value="without jewelry"),
        gr.Radio(["with tatoo", "without tatoo"], value="without tatoo"),
        gr.Radio(["man", "woman"], value="man"),
        gr.Radio(["child", "adult", "old"], value="adult"),
        gr.Textbox(value="white"),
    ],
    "text",
    title="Biometry data generator",
)
demo.launch(server_name='0.0.0.0', server_port=7878,share=True)

Running on local URL:  http://0.0.0.0:7879


/root/miniconda3/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.42.0, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://a57614ec913ea846a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
